# Requirements

In [ ]:
import numpy as np
import pandas as pd

# Seed for reproducibility
np.random.seed(42)
SEED = 42

# Preprocessing
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

# Evaluation
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, log_loss, mean_absolute_error, mean_squared_error, r2_score

# Hyperparameter Tuning
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# Data Preprocessing and Cleaning

## Preprocessing

The data's first two rows are headers.
The first row contains the configuration numbers, but spaced one apart.
The second row contains the completion time and mistakes under each configuration number column.

In [ ]:
df = pd.read_csv('data.csv')
df.head()

In [ ]:
# Convert df. We can make each row contain the config num, completion time, and mistakes.
# Therefore, the data will contain 3 columns: config_num, completion_time, mistakes
# and the data will multiply the number of rows by the 7 (because there are 7 configs)

converted_df:pd.DataFrame = pd.DataFrame(columns=['nick', 'config_num', 'completion_time', 'mistakes'])

# For each participant
num_participants = df.shape[0] - 1 # -1 because the first two rows are headers

for i in range(num_participants):
    # For each config
    for j in range(1, 8):
        config_num = j-1
        
        # Mistakes and completion time are in the same row
        # But they are in different columns. Each config has 2 columns for mistakes and completion time
        mistakes_col = j * 2
        completion_time_col = j * 2 - 1

        mistakes = df.iloc[i+1, mistakes_col]
        completion_time = df.iloc[i+1, completion_time_col]

        # Add row to converted_df
        converted_df = pd.concat([converted_df, pd.DataFrame([[df.iloc[i+1, 0], config_num, completion_time, mistakes]], columns=['nick', 'config_num', 'completion_time', 'mistakes'])])

        # Remove index
        converted_df = converted_df.reset_index(drop=True)

converted_df.head(14)

In [ ]:
print("Previous shape: ", df.shape)
print("New shape: ", converted_df.shape)

df = converted_df

With our current data, the data uses configurations. However, that does not tell us enough. We can add three new variables (`size`, `color`,  and `position`) to give us more insights about our data.
- The baseline by default has `size: regular`, `color: yellow`, and `position: top`.
- Config 1 and 2 changes the size into `small` and `large`
- Config 3 and 4 changes the color into `blue` and `black`
- Config 5 and 6 changes the position into `sticky` and `bottom`

In [ ]:
# Add new column for size, color, and position. Filled based on config_num

df['size'] = df['config_num'].apply(lambda x: 'regular' if x == 0 or x == 3 or x == 4 else 'small' if x == 1 else 'large')
df['color'] = df['config_num'].apply(lambda x: 'yellow' if x == 0 or x == 1 or x == 2 or x == 5 else 'blue' if x == 3 else 'black' if x == 4 else 'yellow')
df['position'] = df['config_num'].apply(lambda x: 'top' if x == 0 or x == 1 or x == 2 or x == 3 or x == 4 else 'sticky' if x == 5 else 'bottom')

df.head(14)

## Cleaning

The data currently has outliers so we have to remove them from our dataset.

In [ ]:
df_cleaned = df.copy()

# Convert size, color, and position to numerical values (0, 1, 2)
# df_cleaned['size'] = df_cleaned['size'].apply(lambda x: 1 if x == 'small' else 0 if x == 'regular' else 2)
# df_cleaned['color'] = df_cleaned['color'].apply(lambda x: 1 if x == 'blue' else 2 if x == 'black' else 0)
# df_cleaned['position'] = df_cleaned['position'].apply(lambda x: 1 if x == 'sticky' else 2 if x == 'top' else 0)

# Remove outliers based on completion time (and possibly other features, just edit the features_of_interest)
features_of_interest = ['completion_time']
clf = IsolationForest(max_samples=100, random_state=SEED)
clf.fit(df_cleaned[features_of_interest])
df_cleaned['anomaly'] = clf.predict(df_cleaned[features_of_interest])

# Plot the data to reveal outliers in completion time using histograms
# Make anomalies red x's and normal data blue dots
completion_times = np.random.normal(loc=50, scale=10, size=1000)

# Create a histogram with adjusted x-axis labels and outliers in red
plt.figure(figsize=(10, 6))

# Plot histogram for normal data in blue
plt.hist(completion_times, bins=20, alpha=0.5, color='blue', edgecolor='black', label='Normal')

# Detect outliers using IQR or other methods (replace this with your outlier detection code)
Q1 = np.percentile(completion_times, 25)
Q3 = np.percentile(completion_times, 75)
IQR = Q3 - Q1
outlier_threshold = 1.5 * IQR
outliers = completion_times[(completion_times < Q1 - outlier_threshold) | (completion_times > Q3 + outlier_threshold)]

# Plot histogram bars for outliers in red
plt.hist(outliers, bins=20, alpha=0.5, color='red', edgecolor='black', label='Outliers')

plt.xlabel('Completion Time')
plt.ylabel('Frequency')
plt.title('Distribution of Completion Time')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.legend()
plt.tight_layout()  # Adjust layout for better spacing
plt.show()

# Filter out outliers based on completion time
df_cleaned = df_cleaned[df_cleaned['anomaly'] == 1]

# Drop the 'anomaly' column
df_cleaned = df_cleaned.drop('anomaly', axis=1)

print("Previous shape:", df.shape)
print("New shape after removing outliers based on completion time:", df_cleaned.shape)

df_cleaned.head(14)


# Model

## Initial Preprocessing

First we need to convert our initial categorical features into numerical values.

In [ ]:
# Convert size, color, and position to numerical values (0, 1, 2)
df_cleaned['size'] = df_cleaned['size'].apply(lambda x: 1 if x == 'small' else 0 if x == 'regular' else 2)
df_cleaned['color'] = df_cleaned['color'].apply(lambda x: 1 if x == 'blue' else 2 if x == 'black' else 0)
df_cleaned['position'] = df_cleaned['position'].apply(lambda x: 1 if x == 'sticky' else 2 if x == 'top' else 0)

To help us decide whether we would keep the mistakes column as a feature of the our model, we first check how many mistakes people made in our experiment.

In [ ]:
mistakes_num = df_cleaned['mistakes'].value_counts()
mistakes_num

In [ ]:
mistake_rate = mistakes_num[1] / (mistakes_num[0] + mistakes_num[1])
mistake_rate

Since the mistake rate is low, we can leave it out. The features of the model are the size, color, and position, and the predicted value is the completion time. The config_num is also disregarded, as it is redundant. It can be derived from the unique combination of size, color, and position. This avoids multicollinearity in our model.

We can now begin splitting the data for our models to use. We conduct an 80-20 train-test split. Validation split is no longer needed because it's done by the model.

In [ ]:
# Split data into train, and test sets. Goal is completion time
X = df_cleaned[['size', 'color', 'position']]
y = df_cleaned['completion_time']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Count number of samples per split
print("Total number of samples:", len(X))
print("Number of samples in training set:", len(X_train))
print("Number of samples in test set:", len(X_test))

## Model Training

### Training

We call RandomForestRegressor from scikitlearn and fit our training data.  

In [ ]:
rf = RandomForestRegressor()

rf.fit(X_train, y_train)

### Evaluation

In [ ]:
train_predictions = rf.predict(X_train)
test_predictions = rf.predict(X_test)

After making the predictions, we evaluate our model using the mean absolute error (MAE), mean squared error (MSE), R^2, and mean absolute percentage error (MAPE) metrics.

In [ ]:
# Convert 'y_test' and 'y_test_pred' to numeric types
y_test = pd.to_numeric(y_test)
test_predictions = pd.to_numeric(test_predictions)

# Calculate evaluation metrics for the test set
mae_test = mean_absolute_error(y_test, test_predictions)
mse_test = mean_squared_error(y_test, test_predictions)
r2_test = r2_score(y_test, test_predictions)
mape = np.mean(np.abs((y_test - test_predictions) / np.abs(y_test))) 

# Print or log the evaluation metrics for the test set
print(f'Test MAE: {mae_test:.4f}')
print(f'Test MSE: {mse_test:.4f}')
print(f'Test R^2 Score: {r2_test:.4f}')
print(f'Mean Absolute Percentage Error (MAPE): {mape * 100:.4f}')

To provide additional context to the MAE metric, we get the range of the completion times. 

In [ ]:
df_cleaned['completion_time'] = df_cleaned['completion_time'].astype(float)
max_time = df_cleaned['completion_time'].max()
min_time = df_cleaned['completion_time'].min()
min_time, max_time

### Findings

The relatively low MAE is indicates that on average, the model's predictions are pretty close to the actual values. This is a relatively low value as the range of the predicted value (completion time) is from 21.826 to 50.157.

The MSE metric is fairly high. This may be due to the limited number of samples in the dataset, where the model is not able to identify key patterns in the data to be able to make predictions effectively.

The low R^2 score suggests that the model explains a small portion of variability in the target variable.

The MAPE value indicates that on average, the predictions are 15.71% off from the actual values.



## Hyperparameter Tuning

We use GridSearchCV to find the best parameters for the model that would lead to the best results.

### Hyperparameter Searching (GridSearchCV)

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=5)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
best_params = grid_search.best_params_
best_params

The above parameters turned out to be optimal for our model. 

### Model Training

We train a new model with our optimal parameters.

In [ ]:
tuned_rf = RandomForestRegressor(**best_params)
tuned_rf.fit(X_train, y_train)

In [ ]:
train_predictions = tuned_rf.predict(X_train)
test_predictions = tuned_rf.predict(X_test)

### Evaluation

In [ ]:
# Convert 'y_test' and 'y_test_pred' to numeric types
y_test = pd.to_numeric(y_test)
test_predictions = pd.to_numeric(test_predictions)

# Calculate evaluation metrics for the test set
mae_test = mean_absolute_error(y_test, test_predictions)
mse_test = mean_squared_error(y_test, test_predictions)
r2_test = r2_score(y_test, test_predictions)
mape = np.mean(np.abs((y_test - test_predictions) / np.abs(y_test))) 

# Print or log the evaluation metrics for the test set
print(f'Test MAE: {mae_test:.4f}')
print(f'Test MSE: {mse_test:.4f}')
print(f'Test R^2 Score: {r2_test:.4f}')
print(f'Mean Absolute Percentage Error (MAPE): {mape * 100:.4f}')

### Findings

The evaluation metrics of the tuned model is very similar to the performance of the original model. The MAE and MSE are slightly higher in the tuned model than in the original model, which indicates that the tuned model are slightly further from the actual values.

The R^2 score is slightly lower for the tuned model, which suggests that its explains less variance in the target variable compared to the original model.

The MAPE is slightly lower for the tuned model, indicating that the predicted completion times are slightly closer to the actual values.

Overall, the tuning process did not significantly improve the model's performance. One possible reason is that the model was already performing at its maximum performance even before the tuning. The nature of the dataset such as the features and the number of samples is also a possible reason for the insignficant difference in model performance before and after the tuning. 

# Conclusion